In [1]:
%%writefile Test_21.py
import pandas as pd
import pymysql
from tabulate import tabulate
import streamlit as st
import matplotlib.pyplot as plt

providers = pd.read_csv('C:/Users/itzme/Downloads/providers_data.csv')
receivers = pd.read_csv('C:/Users/itzme/Downloads/receivers_data.csv')
food_listings = pd.read_csv('C:/Users/itzme/Downloads/food_listings_data.csv')
claims_data = pd.read_csv('C:/Users/itzme/Downloads/claims_data.csv')

connection = pymysql.connect(
    host='localhost',
    user='root',
    password='root',
    database='employee'
)

#print("MySQL Database Connected Successfully")

cursor = connection.cursor()

cursor.execute('drop table if exists employee.provider_data')
cursor.execute('drop table if exists employee.food_listings')
cursor.execute('drop table if exists employee.receivers_data')
cursor.execute('drop table if exists employee.claims_data')

#print("Tables Dropped successfully")

#table creation

cursor.execute("""create table if not exists provider_data (
provider_id int primary key,
Name varchar(255),
type varchar(255),
address varchar(255),
City varchar(255),
Contact varchar(255)
);""")

cursor.execute("""create table if not exists food_listings(
 Food_id int primary key ,
Food_name varchar(255),
Quantity int,
expiry_date varchar(255),
provider_id int,
provider_type varchar(255),
location varchar(255),
Food_type varchar(255),
Meal_type varchar(255));""")

cursor.execute("""create table if not exists claims_data (

claim_id int primary key,
Food_id int,
Receiver_id int,
Status varchar(255),
Timestamp varchar(255));""")

cursor.execute("""create table if not exists Receivers_data (
receiver_id int primary key,
Name varchar(255),
Type varchar(255),
City varchar(255),
Contact varchar(255)
);""")

#print("tables has been created Successfully")

#insert data to table
for index,row in providers.iterrows():
    cursor.execute("""INSERT INTO provider_data (Provider_ID,Name,Type,Address,City,Contact) 
VALUES (%s, %s, %s, %s,%s,%s)""", tuple(row))

for index,row in receivers.iterrows():
     cursor.execute("""Insert into receivers_data(Receiver_ID,Name,Type,City,Contact)
values(%s,%s,%s,%s,%s)""", tuple(row))

for index,row in food_listings.iterrows():
    cursor.execute("""Insert into food_listings(Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type)
values(%s,%s,%s,%s,%s,%s,%s,%s,%s)""", tuple(row))

for index,row in claims_data.iterrows():
    cursor.execute("""Insert into claims_data(Claim_ID,Food_ID,Receiver_ID,Status,Timestamp)
values(%s,%s,%s,%s,%s)""", tuple(row))

connection.commit()
#print("Data successfully inserted into the MySQL table!")

#project introduction
selected_tab = st.sidebar.radio("Choose a section:", ["User Introduction","Project_introduction","CRUD_Operations", "View tables", "SQL Q&A","Analysis & Visualizations",
"Learner SQL Questions"])

if selected_tab == 'Project_introduction':
    st.markdown("""
    <style>
        @keyframes fadeIn {
            0% { opacity: 0; }
            100% { opacity: 1; }
        }
        .title {
            animation: fadeIn 2s ease-in-out;
            font-size: 48px;
            font-weight: bold;
            text-align: center;
            color: #ff6f61;
        }
    </style>
    <h1 class="title">Local Food Waste Management</h1>
""", unsafe_allow_html=True)

#st.image("https://mir-s3-cdn-cf.behance.net/project_modules/disp/c4407656022977.599d3d9d7d1b7.gif",width=200)

#st.sidebar.selectbox("List of Operations",options=["CRUD Operations","View Tables"])   
    st.markdown("""This platform helps reduce food waste by connecting providers and receivers efficiently.
    Analyze food donation trends and optimize distribution strategies with real-time insights.
    Discover how communities can work together to minimize waste and maximize impact!""")

    st.write("This dataset contains food provider details, receiver claims, food listings, and transaction records.")

    col1,col2=st.columns(2)
    with col1:
        st.image("https://png.pngtree.com/png-clipart/20200401/original/pngtree-food-waste-colorful-doodle-drawing-with-text-png-image_5337976.jpg",width=200)
    with col2:
        st.image("https://mir-s3-cdn-cf.behance.net/project_modules/disp/c4407656022977.599d3d9d7d1b7.gif",width=200)

#view table module 

if selected_tab == 'View tables':
    st.header("Select the table to view the records")
    table_selection=st.selectbox("Table name",["Select tablename","provider_data","receivers_data","food_listings","claims_data"])
    
    if table_selection != "Select tablename":
        SQL_Query = f"select * from {table_selection}"
        cursor.execute(SQL_Query)
        table_data = cursor.fetchall()
        column_headers = [desc[0] for desc in cursor.description]

        table_df = pd.DataFrame(table_data, columns=column_headers)
        st.dataframe(table_df) 
    else:
        st.write("Please select a table to view records.")
# View Tables module completed

# SQL Q & A
if selected_tab =="SQL Q&A":
    st.header("SQL questions")
    cursor.execute("SELECT Question FROM SQL_Questions")
    question_list = [q[0] for q in cursor.fetchall()]
    selected_question = st.selectbox("Select a question:", ["Select a question"] + question_list)

    query_dict = {
        "How many food providers and receivers are there in each city?": 
            """SELECT a.city, COUNT(a.city) as count FROM provider_data a INNER JOIN receivers_data b ON a.city = b.city GROUP BY a.city
             order by count desc limit 10""",
        
        "Which type of food provider (restaurant, grocery store, etc.) contributes the most food?": 
            "select count(*),Provider_type from food_listings group by Provider_type order by count(provider_type) desc",
        
        "What is the contact information of food providers in a specific city?": 
            "select City from provider_data",
        
        "Which receivers have claimed the most food?": 
            """select count(*), a.Name  from receivers_data a inner join claims_data b on a.receiver_id = b.receiver_id where b.status ='completed'
             group by a.name order by count(*) desc""",
        
        "What is the total quantity of food available from all providers?": 
            "select sum(a.quantity),b.type from food_listings a inner join provider_data b on a.provider_id = b.provider_id group by b.type",
        
        "Which city has the highest number of food listings?":"select location,Count(*) as Count from food_listings group by location order by count desc limit 10",

        "What are the most commonly available food types?":"select Food_type, count(*) as count from food_listings group by food_type order by count desc",

        " How many food claims have been made for each food item?":"SELECT food_id, COUNT(*) AS claim_count FROM claims_data GROUP BY food_id ORDER BY claim_count DESC",
        
        "What percentage of food claims are completed vs. pending vs. canceled?":"""select   status, count(*) * 100 / (select count(*) from claims_data) as percentage 
        from claims_data group by status""",

        "What is the average quantity of food claimed per receiver?":"""select Avg(a.quantity) as average, b.receiver_id from food_listings a inner join claims_data b on a.food_id = b.food_id
                        group by b.receiver_id order by average desc limit""",

        "Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?":"""select a.meal_type, COUNT(b.claim_id) AS claim_count from food_listings a inner join claims_data b on a.food_id = b.food_id 
        group BY a.meal_type order by claim_count desc""",

        "What is the total quantity of food donated by each provider?":"""select provider_id,sum(Quantity) as total_quantity from food_listings group by provider_id order by total_quantity desc"""


    }
    
    if selected_question in query_dict:
        SQL_Query = query_dict[selected_question]
        cursor.execute(SQL_Query)
        table_data = cursor.fetchall()
        column_headers = [desc[0] for desc in cursor.description]
        table_df = pd.DataFrame(table_data, columns=column_headers)

        #selected_city = st.selectbox("Select City:", ["All"] + list(table_df["City"].unique()))
        st.dataframe(table_df)
    else:
        st.write("Please select a question to retrieve SQL results.")
#CRUD OPERATIONS

if selected_tab == 'CRUD_Operations':
    select_table=st.selectbox("select table from below",["provider_data","receiver_data","claims_data","food_listings"])
    select_operation=st.selectbox("Select Operation to perform",["Insert","Update","Delete"])

    
    if select_table == "provider_data":
        provider_id = st.text_input("ID")
        name = st.text_input("Provider Name")
        type = st.selectbox("Provider Type", ["Restaurant", "Grocery Store", "Farm", "Bakery"])
        address = st.text_input("Address")
        city = st.text_input("City")
        contact_info = st.text_input("Contact Info")

        if select_operation == "Insert":
            if st.button("Add Provider"):
                insert_query1 = f"""INSERT INTO provider_data (provider_id, name, type, address, city, contact)
                VALUES ('{provider_id}', '{name}', '{type}', '{address}', '{city}', '{contact_info}');"""
                cursor.execute(insert_query1)
                connection.commit()
                st.success("Record has been inserted successfully!")

        elif select_operation == "Update":
            if st.button("Update Provider"):
                update_query = f"""UPDATE provider_data SET contact = '{contact_info}', city = '{city}' WHERE provider_id = '{provider_id}'"""
                cursor.execute(update_query)
                connection.commit()
                st.success("Record has been updated successfully!")

        elif select_operation == "Delete":
            if st.button("Delete Provider"):
                delete_query = f"DELETE FROM provider_data WHERE provider_id = '{provider_id}'"
                cursor.execute(delete_query)
                connection.commit()
                st.success("Record has been deleted successfully!")

    elif select_table == "receiver_data":  
        receiver_id = st.text_input("ID")
        name = st.text_input("Receiver Name")
        type = st.selectbox("Receiver Type", ["NGO", "Shelter", "Individual", "Charity"])
        city = st.text_input("City")
        contact = st.text_input("Contact Info")

        if select_operation == "Insert":
            if st.button("Add Receiver"): 
                insert_query1 = f"""INSERT INTO receivers_data (receiver_id, name, type, city, contact)
                VALUES ('{receiver_id}', '{name}', '{type}', '{city}', '{contact}');"""
                cursor.execute(insert_query1)
                connection.commit()
                st.success("Record has been inserted successfully!")

        elif select_operation == "Update":
            if st.button("Update Receiver"):
                update_query = f"""UPDATE receivers_data SET contact = '{contact}', city = '{city}' WHERE receiver_id = '{receiver_id}'"""
                cursor.execute(update_query)
                connection.commit()
                st.success("Record has been updated successfully!")

        elif select_operation == "Delete":
            if st.button("Delete Receiver"): 
                delete_query = f"DELETE FROM receivers_data WHERE receiver_id = '{receiver_id}'"
                cursor.execute(delete_query)
                connection.commit()
                st.success("Record has been deleted successfully!")

   
    elif select_table == "claims_data":  
        claim_id = st.text_input("ID")
        food_id = st.text_input("food id")
        receiver_id = st.text_input("receiver id")
        status = st.selectbox("Status", ["Pending", "Completed", "Cancelled"])
        timestamp = st.text_input("M/D/YYYY HH:MM")

        if select_operation == "Insert":
            if st.button("Add Claim"): 
                insert_query1 = f"""INSERT INTO Claims_data (claim_id,food_id,receiver_id,status, timestamp)
                VALUES ('{claim_id}', '{food_id}', '{receiver_id}', '{status}', '{timestamp}');"""
                cursor.execute(insert_query1)
                connection.commit()
                st.success("Record has been inserted successfully!")

        elif select_operation == "Update":
            if st.button("Update Receiver"):
                update_query = f"""UPDATE claims_data SET status = '{status}' WHERE claim_id = '{claim_id}'"""
                cursor.execute(update_query)
                connection.commit()
                st.success("Record has been updated successfully!")

        elif select_operation == "Delete":
            if st.button("Delete claim"): 
                delete_query = f"DELETE FROM claims_data WHERE claim_id = '{claim_id}'"
                cursor.execute(delete_query)
                connection.commit()
                st.success("Record has been deleted successfully!")    

  
    elif select_table == "food_listings":  
        food_id = st.text_input("Food_id")
        food_name = st.text_input("food name")
        Quantity = st.text_input("Quantity")
        expiry_date = st. text_input("M/D/YYYY")
        provider_id = st.text_input("provider_id")
        provider_type = st.selectbox("provider type", ["Grocery Store", "Catering Service", "Restaurant","Supermarket"])
        location = st.text_input("Location")
        Food_type =st.selectbox("Food type", ["Vegeterian", "Vegan", "Non-vegeterian"])
        Meal_type = st.selectbox("Meal type", ["Breakfast", "Lunch", "Dinner","Snacks"])

        if select_operation == "Insert":
            if st.button("Add Food_listings"): 
                insert_query1 = f"""INSERT INTO food_listings (food_id,food_name,Quantity,expiry_date,provider_id,provider_type,location,Food_type,Meal_type)
                VALUES ('{food_id}', '{food_name}', '{Quantity}', '{expiry_date}','{provider_id}','{provider_type}','{location}','{Food_type}','{Meal_type}');"""
                cursor.execute(insert_query1)
                connection.commit()
                st.success("Record has been inserted successfully!")

        elif select_operation == "Update":
            if st.button("Update Food Data"):
                update_query = f"""UPDATE Food_listings SET Food_name = '{food_name}',Quantity = '{Quantity}',expiry_date ='{expiry_date}',
                provider_id ='{provider_id}',provider_type = '{provider_type}',location = '{location}' ,food_type = '{Food_type}',
                Meal_type = '{Meal_type}' WHERE Food_id = '{food_id}'"""
                cursor.execute(update_query)
                connection.commit()
                st.success("Record has been updated successfully!")

        elif select_operation == "Delete":
            if st.button("Delete Food Listings"): 
                delete_query = f"DELETE FROM Food_listings WHERE Food_id = '{food_id}'"
                cursor.execute(delete_query)
                connection.commit()
                st.success("Record has been deleted successfully!")    

if selected_tab == "Analysis & Visualizations":
    st.header("Analysis & Visualizations")
    query_dict = {
        "How many food providers and receivers are there in each city?": 
            """SELECT a.city, COUNT(a.city) as count FROM provider_data a INNER JOIN receivers_data b ON a.city = b.city GROUP BY a.city
             order by count desc limit 10""",
        
        "Which type of food provider (restaurant, grocery store, etc.) contributes the most food?": 
            "select count(*) as count,Provider_type from food_listings group by Provider_type order by count(provider_type) desc",
        
        "Which receivers have claimed the most food?": 
            """select count(*) as count, a.Name  from receivers_data a inner join claims_data b on a.receiver_id = b.receiver_id where b.status ='completed'
             group by a.name order by count(*) desc limit 15""",
        
        "What is the total quantity of food available from all providers?": 
            "select sum(quantity) as total_quantity  from food_listings",
        
        "Which city has the highest number of food listings?":"select location,Count(*) as Count from food_listings group by location order by count desc limit 10",

        "What are the most commonly available food types?":"select Food_type, count(*) as count from food_listings group by food_type order by count desc",

        "How many food claims have been made for each food item?":"SELECT a.food_id,b.food_name ,COUNT(*)  as claim_count  FROM claims_data a inner join food_listings b on a.food_id = b.food_id GROUP BY a.food_id ORDER BY claim_count DESC",
        
        "What percentage of food claims are completed vs. pending vs. canceled?":"""select   status, count(*) * 100 / (select count(*) from claims_data) as percentage 
        from claims_data group by status""",

        "What is the average quantity of food claimed per receiver?":"""select Avg(a.quantity) as average, b.receiver_id, c.name as Receiver_name from food_listings a inner join claims_data b on a.food_id = b.food_id
        inner join receivers_data c on b.receiver_id = c.receiver_id group by b.receiver_id order by average desc limit 20""",

        "Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?":"""select a.meal_type, COUNT(b.claim_id) AS claim_count from food_listings a inner join claims_data b on a.food_id = b.food_id 
        group BY a.meal_type order by claim_count desc""",

        "What is the total quantity of food donated by each provider?":"""select distinct(b.type),sum(a.Quantity) as total_quantity from food_listings a inner join provider_data b on a.provider_id = b.provider_id
        group by b.type order by total_quantity desc"""


    }
    # Select a question for visualization
    selected_question = st.selectbox("Select a question for analysis:", ["Select a question"] + list(query_dict.keys()))

    if selected_question in query_dict:
        SQL_Query = query_dict[selected_question]
        cursor.execute(SQL_Query)
        table_data = cursor.fetchall()
        column_headers = [desc[0] for desc in cursor.description]
        table_df = pd.DataFrame(table_data, columns=column_headers)
        st.dataframe(table_df)

        if selected_question == "How many food providers and receivers are there in each city?":
            st.subheader("Food Providers & Receivers Per City")
            table_df.set_index("city",inplace=True)
            st.bar_chart(table_df)
            
        elif selected_question == "Which type of food provider (restaurant, grocery store, etc.) contributes the most food?":
            st.subheader("Contributions based on Provider Type")
            table_df.set_index("Provider_type",inplace=True)
            st.scatter_chart(table_df["count"])

        elif selected_question == "Which receivers have claimed the most food?":
            st.subheader("Receivers that claimed Most food")
            table_df.set_index("Name",inplace=True)
            st.bar_chart(table_df["count"])

        elif selected_question == "Which city has the highest number of food listings?":
            st.subheader("Highest number of food listings")
            table_df.set_index("location",inplace=True)
            st.area_chart(table_df["Count"])
        
        elif selected_question == "What are the most commonly available food types?":
            st.subheader("Highest number of food listings")
            table_df.set_index("Food_type",inplace=True)
            st.bar_chart(table_df["count"])

        elif selected_question == "How many food claims have been made for each food item?":
            st.subheader("Food Claim made for each food")
            table_df.set_index("food_name",inplace=True)
            st.bar_chart(table_df["claim_count"])

        elif selected_question == "What percentage of food claims are completed vs. pending vs. canceled?":
            st.subheader("Highest number of food listings")
            table_df.set_index("status",inplace=True)
            st.scatter_chart(table_df["percentage"])
            
        elif selected_question == "What is the average quantity of food claimed per receiver?":
            st.subheader("Average Quantity of Food claim per receiver")
            table_df.set_index("Receiver_name",inplace=True)
            st.line_chart(table_df["average"])

        elif selected_question == "What is the total quantity of food donated by each provider?":
            st.subheader("Total Quantity of food by each provider")
            table_df.set_index("type",inplace=True)
            st.scatter_chart(table_df["total_quantity"])

if selected_tab == "Learner SQL Questions":
    st.markdown("Learner SQL Question")
    Query_Dict2 = {"Find the count of Food that is crossing the expiry date ?":"""select count(*) as count,b.food_name from claims_data a inner join food_listings b on a.food_id = b.food_id
                    where CONVERT(STR_TO_DATE(a.timestamp, '%m/%d/%Y %H:%i'), DATE) >= CONVERT(STR_TO_DATE(b.expiry_date, '%m/%d/%Y'), DATE) group by b.food_name """,

                  "How many Claim request received per day ?":"""select  CONVERT(STR_TO_DATE(timestamp, '%m/%d/%Y %H:%i'),Date) as Date,count(*) as Claim_request from claims_data 
                    group by CONVERT(STR_TO_DATE(timestamp, '%m/%d/%Y %H:%i'),Date) order by 2 desc""",
                    
                  "Find the type of donaters and contribution count for each donater type?":"select type,count(*) as count from receivers_data group by type",
                  
                  "how many claimed food that are expired?":"""select count(*) as count,food_name from claims_data a inner join food_listings b on a.food_id = b.food_id
                        where CONVERT(STR_TO_DATE(a.timestamp, '%m/%d/%Y %H:%i'), DATE) >= CONVERT(STR_TO_DATE(b.expiry_date, '%m/%d/%Y'), DATE) and a.status ='completed'
                        group by b.food_name""",
                        
                 "which provider food has high expired food quantity?":"""select sum(a.quantity) as quantity,a.provider_type from food_listings a inner join claims_data b on a.food_id = b.food_id
where convert(str_to_date(b.timestamp, '%m/%d/%Y %H:%i'),Date)>= convert(str_to_date(a.expiry_date, '%m/%d/%Y'), date) 
group by a.provider_type"""}
      
      
    select_query2 = st.selectbox("Select Question for analysis:",['Select a Question']+list(Query_Dict2.keys()))
                
    if select_query2 in Query_Dict2:
        Query = Query_Dict2[select_query2]
        cursor.execute(Query)
        table_data = cursor.fetchall()
        column_headers= [desc[0] for desc in cursor.description]
        df = pd.DataFrame(table_data,columns = column_headers)
        st.dataframe(df)
        
        if select_query2 == 'Find the count of Food that is crossing the expiry date ?':
            st.subheader('count of food that is expired')
            df.set_index("food_name",inplace =True)
            st.bar_chart(df['count'])

        elif select_query2 =='How many Claim request received per day ?':
            st.subheader('Claim Request received per day')
            df.set_index("Date",inplace = True)
            st.bar_chart(df['Claim_request'])

        elif select_query2 =='Find the type of donaters and contribution count for each donater type?':
            st.subheader('contribution based on donater type')
            df.set_index('type',inplace = True)
            st.line_chart(df['count'])

        elif select_query2 =='how many claimed food that are expired?':
            st.subheader('Claimed Food that is Expired')
            df.set_index("food_name",inplace = True)
            st.bar_chart(df['count'])
        
        elif select_query2 == 'which provider food has high expired food quantity?':
            st.subheader('Provider that donates Expired or near Expired food')
            df.set_index("provider_type",inplace= True)
            st.bar_chart(df['quantity'])


if selected_tab == 'User Introduction':
    st.header("Self Introduction")
    st.subheader("Welcome to Local Food waste management Project")
    st.markdown("""Hello,My Name is Raghuraman and i am passionate about Data Analysis, Automations and Data visualizations etc.
                the Project Food waste Management consists of 4 Datasets where i was able analyse multiple points from Food donaters,
                Food Wastage,food Claim trends.By Leveraging SQL and Python techniques this project delivers improved decision making,
                seamless Data processing.""")

    st.image("https://cdn.pixabay.com/photo/2023/11/19/06/07/business-8398064_1280.jpg",width =500)
            


cursor.close()
connection.close()


Overwriting Test_21.py


In [2]:
!streamlit run Test_21.py

^C
